In [46]:
import time
import requests
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from collections import defaultdict
from itertools import combinations
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network

In [2]:
opportunities = pd.read_pickle('../Data/2022-10-24.pkl')

In [4]:
index = 'URL'
assert opportunities[index].value_counts().max() == 1

In [5]:
discipline = opportunities[[index, 'DisciplineNames']].explode('DisciplineNames')

In [6]:
subject_name = opportunities[[index, 'SubjectNames']].explode('SubjectNames')

In [7]:
discipline

,URL,DisciplineNames
0,https://www.findaphd.com/phds/project/swbio-dt...,Biological Sciences
0,https://www.findaphd.com/phds/project/swbio-dt...,Mathematics
0,https://www.findaphd.com/phds/project/swbio-dt...,Veterinary Sciences
1,https://www.findaphd.com/phds/project/carbon-s...,Biological Sciences
1,https://www.findaphd.com/phds/project/carbon-s...,Environmental Sciences
...,...,...
5519,https://www.findaphd.com/phds/programme/the-in...,Sport & Exercise Science
5520,https://www.findaphd.com/phds/program/phds-at-...,Literature & Culture
5520,https://www.findaphd.com/phds/program/phds-at-...,Computer Science
5520,https://www.findaphd.com/phds/program/phds-at-...,Languages


In [18]:
opportunities.sample(10)

,Title,Description,URL,Deadline,SearchDate,DisciplineIds,DisciplineNames,SubjectIds,SubjectNames,LocationCountryName,LocationCityName,ProgrammeTypes,IId,InstitutionName,DId,DepartmentName,FundingTypes
2648,An integrated predictive tool for City-scale C...,"Supervisory Team. Zhengtong Xie, Steven Herr...",https://www.findaphd.com/phds/project/an-integ...,NaT,2022-10-24,12,Engineering,264,Other,United Kingdom,Southampton,PhD Research Project,349,University of Southampton,1427,Faculty of Engineering and Physical Sciences,"(SF, UK)"
4832,"Tigers in Film: Past, Present and Future Persp...",Aim. The Indian tiger (Panthera tigris tigris)...,https://www.findaphd.com/phds/project/tigers-i...,NaT,2022-10-24,"(19, 7, 9)","(Communication & Media Studies, Creative Arts ...","(182, 208, 315, 321)","(Asian History, Film Studies, History, Product...",United Kingdom,Reading,PhD Research Project,345,University of Reading,1344,School of Archaeology,SF
5251,Strategic synthesis of magnetically interestin...,Molecule-based magnets are highly interesting ...,https://www.findaphd.com/phds/project/strategi...,NaT,2022-10-24,"(12, 24)","(Engineering, Materials Science)","(236, 374)","(Chemical Engineering, Other)",United Kingdom,Bradford,PhD Research Project,285,University of Bradford,4320,Faculty of Life Sciences,SF
1724,Blast noise control,We are looking for a PhD candidate to work on ...,https://www.findaphd.com/phds/project/blast-no...,NaT,2022-10-24,"(12, 25, 29)","(Engineering, Mathematics, Physics)","(230, 264, 378, 387, 431, 447)","(Acoustics, Acoustics Engineering, Data Analys...",United Kingdom,Manchester,PhD Research Project,347,University of Salford,1367,"( Engineering and Environment, School of Science)","(SF, UK)"
1042,Machine-learning-guided design of efficient su...,"The Scientific Mission. Fuel cells, photovolta...",https://www.findaphd.com/phds/project/machine-...,NaT,2022-10-24,"(12, 24, 29, 6)","(Chemistry, Engineering, Materials Science, Ph...","(166, 255, 374, 446)","(Computational Chemistry, Nanotechnology, Othe...",United Kingdom,Warwick,PhD Research Project,369,University of Warwick,1574,Department of Chemistry,"(EU, NE, SF, UK)"
5207,Space Situational Awareness,Space debris is defined as all human-made obje...,https://www.findaphd.com/phds/project/space-si...,NaT,2022-10-24,"(12, 29)","(Engineering, Physics)","(231, 445)","(Aerospace Engineering, Space Science)",New Zealand,Auckland,PhD Research Project,830,University of Auckland,5621,Faculty of Engineering,"(EU, NE, SF, UK)"
4482,Development of chemical tools and bioactive co...,Diversity-oriented synthesis (DOS) is a modern...,https://www.findaphd.com/phds/project/developm...,NaT,2022-10-24,"(26, 4)","(Biological Sciences, Medicine)","(119, 124, 125, 150, 402, 403)","(Biochemistry, Cancer Biology, Cell Biology, O...",United Kingdom,Bradford,PhD Research Project,285,University of Bradford,4320,Faculty of Life Sciences,SF
2238,PhD Studentship in Polymer Tribology in Cryoge...,Applications are invited for a research studen...,https://www.findaphd.com/phds/project/phd-stud...,NaT,2022-10-24,"(12, 24)","(Engineering, Materials Science)","(231, 233, 236, 252, 253, 259, 372)","(Aerospace Engineering, Automotive Engineering...",United Kingdom,London,PhD Research Project,112,Imperial College London,904,Department of Mechanical Engineering,"(EU, NE, SF, UK)"
3008,Role of diet to combat Heart Failure and Strok...,Heart Failure (HF) and stoke affects significa...,https://www.findaphd.com/phds/project/role-of-...,NaT,2022-10-24,"(26, 4)","(Biological Sciences, Medicine)","(139, 140, 395, 398)","(Epidemiology, Molecular Biology, Molecular Ge...",United Kingdom,Liverpool,PhD Research Project,326,University of Liverpool,2924,Institute of Ageing and Chronic Disease Depart...,SF
913,Intuitive imitation learning for robots,Imitation learning (IL) or Learning from Demon...,https://www.findaphd.com/phds/project/intuitiv...,NaT,2022-10-24,12,Engineering,"(243, 258, 264)","(Electronic Engineering,

In [164]:
counts = defaultdict(int)
for index, row in opportunities.iterrows():
    names = []
    for col in ['SubjectNames', 'DisciplineNames']:
        if isinstance(row[col], tuple):
            names.extend(list(row[col]))
        else:
            names.append(row[col])
    names = [i for i in names if i not in ['', 'Other']]
        
    for pair in combinations(names, 2):
        pair = tuple(sorted(pair))  
        counts[pair] += 1
        
counts = pd.Series(counts).reset_index()
counts = counts.loc[counts[0] > 20]
maxCount = counts[0].max()
counts = counts.apply(tuple, axis=1).to_list()

In [165]:
G = nx.Graph()
G.add_weighted_edges_from(counts)

In [169]:
topic = 'Evolution'
subGraph = nx.ego_graph(G, topic, radius=1)
for u, v, d in subGraph.edges(data=True):
    d['weight'] = (d['weight'] / maxCount) * 3
nt = Network('500px', '500px')
nt.show_buttons(filter_=['physics']) 
nt.from_nx(subGraph)
nt.show(f'nx-{topic}.html')

In [168]:
counts

[('Bacteriology', 'Bioinformatics', 24),
 ('Bacteriology', 'Microbiology', 47),
 ('Bacteriology', 'Biological Sciences', 64),
 ('Biodiversity', 'Bioinformatics', 25),
 ('Biodiversity', 'Environmental Biology', 35),
 ('Biodiversity', 'Evolution', 25),
 ('Biodiversity', 'Genomics', 25),
 ('Biodiversity', 'Biological Sciences', 79),
 ('Bioinformatics', 'Data Analysis', 93),
 ('Bioinformatics', 'Environmental Biology', 21),
 ('Bioinformatics', 'Evolution', 96),
 ('Bioinformatics', 'Genomics', 96),
 ('Bioinformatics', 'Microbiology', 97),
 ('Bioinformatics', 'Biological Sciences', 525),
 ('Bioinformatics', 'Mathematics', 120),
 ('Biological Sciences', 'Data Analysis', 169),
 ('Data Analysis', 'Mathematics', 385),
 ('Environmental Biology', 'Evolution', 21),
 ('Environmental Biology', 'Microbiology', 31),
 ('Biological Sciences', 'Environmental Biology', 143),
 ('Evolution', 'Genomics', 48),
 ('Evolution', 'Microbiology', 56),
 ('Biological Sciences', 'Evolution', 206),
 ('Evolution', 'Mathe

In [97]:
nt = Network('500px', '500px')
nt.from_nx(G)
nt.show('nx.html')

In [129]:
len(subGraph.edges())

2359

In [131]:
for node in subGraph.edges():
    print(node)

('Bacteriology', 'Bioinformatics')
('Bacteriology', 'Environmental Biology')
('Bacteriology', 'Genomics')
('Bacteriology', 'Microbiology')
('Bacteriology', 'Biological Sciences')
('Bacteriology', 'Chemistry')
('Bacteriology', 'Molecular Biology')
('Bacteriology', 'Molecular Genetics')
('Bacteriology', 'Biochemistry')
('Bacteriology', 'Genetics')
('Bacteriology', 'Medicine')
('Bioinformatics', 'Biodiversity')
('Bioinformatics', 'Data Analysis')
('Bioinformatics', 'Environmental Biology')
('Bioinformatics', 'Evolution')
('Bioinformatics', 'Genomics')
('Bioinformatics', 'Microbiology')
('Bioinformatics', 'Biological Sciences')
('Bioinformatics', 'Mathematics')
('Bioinformatics', 'Veterinary Sciences')
('Bioinformatics', 'Cell Biology')
('Bioinformatics', 'Genetics')
('Bioinformatics', 'Medical Physics')
('Bioinformatics', 'Molecular Biology')
('Bioinformatics', 'Zoology')
('Bioinformatics', 'Physics')
('Bioinformatics', 'Molecular Genetics')
('Bioinformatics', 'Parasitology')
('Bioinforma